In [181]:
import pandas as pd
from pandas.api.types import is_object_dtype, is_numeric_dtype
import numpy as np
from tqdm import tqdm
import plotly.express as px
import statsmodels.api as sma
from sklearn import preprocessing 
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest, SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier
from sklearn.metrics import f1_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
data = pd.read_csv("Breast Cancer/data.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

#Dzielimy dane na uczące i testowe 
X = data.drop(['id','diagnosis'], axis=1)
y = data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.3,
                                                   random_state=42)

# Initialize classifier
gbc = GradientBoostingClassifier(max_depth=5, random_state=42)  # Dla problemu klasyfikacji

In [186]:
# INNE PODEJSCIE POD ACO 
# POMYŚLEĆ O SKALOWANIU
# Krok 1 Ranking parametrów
mutual_info = mutual_info_classif(X_train, y_train)
weights = np.array(mutual_info)
print("Wagi:\n",weights)

Wagi:
 [0.32618747 0.10177276 0.37107807 0.32882171 0.0794367  0.20089121
 0.36897949 0.44488804 0.04502213 0.02629505 0.25094202 0.01141171
 0.24998774 0.31697764 0.01688705 0.04094205 0.12616505 0.10974477
 0.02026062 0.00641461 0.4412525  0.1670385  0.45254054 0.43881309
 0.09618387 0.21113197 0.31459994 0.44550612 0.08302258 0.06048958]


In [187]:
# KROK 2
# DEFINIUJE PHEROMONE ARRAY (TAKA SAMA WARTOSC)
def generate_pheromone_array(num_features, value):
    pheromone_array = np.full(num_features, value)
    return pheromone_array

In [188]:
pheromone_array = generate_pheromone_array(30, 0.2)
print("Pheromone_array: \n", pheromone_array)

Pheromone_array: 
 [0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2
 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]


In [ ]:
# Krok 3
# Generacja początkowego ustawienia mrówek
def generate_ant_population(ants_colony_size, num_features):
    ants = np.zeros(shape=(ants_colony_size, num_features), dtype=int)
    for ant in ants:
        random_node_index = np.random.randint(0, num_features)
        ant[random_node_index] = 1
    print("Ants generated!\n")
    return ants

ants = generate_ant_population(30, 30)
print("Ants:\n", ants)

In [180]:
from itertools import accumulate
import operator

# KROK 4 KALKULACJA PRAWDOPODBIENSTWA DLA KANDYDATOW i SELEKCJA NOWEJ CECHY
alpha = 1.0
beta = 1.0

print("Initial Ants Population:\n", ants)

# Implement Roulette Wheel Selection
def roulette_wheel_selection(probabilities, num_candidates):
    cumulative_probabilities = list(accumulate(probabilities, operator.add))
    print(cumulative_probabilities)
    selected_index = np.searchsorted(cumulative_probabilities, np.random.rand()) % num_candidates
    return selected_index

def ants_make_move(ants_population):
    for ant_index in range(len(ants_population)):
        current_ant = ants_population[ant_index]
        
        print("Ant {}:".format(ant_index + 1))
        print(current_ant)
    
        # Calculate the sum of probabilities for normalization
        total_probability = np.sum((pheromone_array[current_ant == 0] ** alpha) * (weights[current_ant == 0] ** beta))
        probabilities_for_each_candidate = np.zeros(len(current_ant))
        
        for j in range(len(current_ant)):
            if current_ant[j] == 0:
                pheromone_level = pheromone_array[j]
                attractiveness = weights[j]
                probability = ((pheromone_level ** alpha) * (attractiveness ** beta)) / total_probability
                probabilities_for_each_candidate[j] = probability
    
        selected_feature_index = roulette_wheel_selection(probabilities_for_each_candidate, len(probabilities_for_each_candidate))
        print("Selected Feature Index:", selected_feature_index)
        print()
        current_ant[selected_feature_index] = 1
        print("Ant selected:\n", current_ant)
        print()

    return ants_population


ants_population_after_move = ants_make_move(ants)

print("New ants population\n:", ants_population_after_move)

Initial Ants Population:
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0

In [183]:
# Krok 5 SPRAWDZ FITNESS MRÓWEK
def calculate_ant_fitness(features):
   feature_mask = features.astype(bool)
   X_train_subset = X_train.loc[:, feature_mask]
   X_test_subset = X_test.loc[:, feature_mask]
   gbc.fit(X_train_subset, y_train)
   preds = gbc.predict(X_test_subset)
   f1_score_subset = round(f1_score(y_test, preds, average='weighted'), 3)
   return f1_score_subset

In [184]:
fitness_scores_ants = [calculate_ant_fitness(features) for features in tqdm(ants_population_after_move, desc='Fitness Score Calculation Progress', colour = "green", leave=True)]
print(fitness_scores_ants)

Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:08<00:00,  3.42it/s]

[0.924, 0.906, 0.959, 0.959, 0.959, 0.959, 0.918, 0.936, 0.866, 0.947, 0.959, 0.93, 0.942, 0.89, 0.965, 0.836, 0.953, 0.901, 0.942, 0.93, 0.941, 0.936, 0.906, 0.906, 0.889, 0.93, 0.929, 0.919, 0.924, 0.93]


In [185]:
# KROK 6 ZAKTUALIZUJ LISTE FEROMONÓW NA PODSTAWIE WYBORÓW MRÓWEK
def update_pheromones(pheromone_array, ants_population, quality_measure, evaporation_rate=0.1):
    pheromone_array *= (1 - evaporation_rate)
    for ant in ants_population:
        for j in range(len(pheromone_array)):
            if ant[j] == 1:
                # Update pheromone levels based on the quality measure
                pheromone_array[j] += quality_measure[j]
    return pheromone_array

updated_pheromones = update_pheromones(pheromone_array, ants_population_after_move, fitness_scores_ants)
print("Updated Pheromone Levels:\n", updated_pheromones)

Updated Pheromone Levels:
 [5.724 1.992 4.016 3.057 1.139 2.098 3.852 2.052 1.912 2.074 4.016 1.11
 5.832 4.63  1.145 1.852 2.086 2.883 0.18  0.18  5.826 2.988 4.71  2.898
 1.958 5.76  3.896 8.451 0.18  0.18 ]


In [ ]:
# KROK 7 WYKONUJEMY KOLEJNE KROKI MRÓWEK

In [198]:
# TEST
# Krok 1 Ranking parametrów
mutual_info = mutual_info_classif(X_train, y_train)


# KROK 2
# DEFINIUJE PHEROMONE ARRAY (TAKA SAMA WARTOSC)
def generate_pheromone_array(num_features, value):
    pheromone_array = np.full(num_features, value)
    return pheromone_array

# Krok 3
# Generacja początkowego ustawienia mrówek
def generate_ant_population(ants_colony_size, num_features):
    ants = np.zeros(shape=(ants_colony_size, num_features), dtype=int)
    for ant in ants:
        random_node_index = np.random.randint(0, num_features)
        ant[random_node_index] = 1
    print("Ants generated!\n")
    return ants

from itertools import accumulate
import operator

# KROK 4 KALKULACJA PRAWDOPODBIENSTWA DLA KANDYDATOW i SELEKCJA NOWEJ CECHY

# Implement Roulette Wheel Selection
def roulette_wheel_selection(probabilities, num_candidates):
    print("Problities:\n", probabilities)
    cumulative_probabilities = list(accumulate(probabilities, operator.add))
    print("Cumulative probabilities: \n", cumulative_probabilities)
    selected_index = np.searchsorted(cumulative_probabilities, np.random.rand()) % num_candidates
    return selected_index

def ants_make_move(ants_population, pheromone_array, weights):
    for ant_index in range(len(ants_population)):
        current_ant = ants_population[ant_index]
        
        print("Ant {}:".format(ant_index + 1))
        print(current_ant)
    
        # Calculate the sum of probabilities for normalization
        total_probability = np.sum((pheromone_array[current_ant == 0] ** alpha) * (weights[current_ant == 0] ** beta))
        probabilities_for_each_candidate = np.zeros(len(current_ant))
        
        for j in range(len(current_ant)):
            if current_ant[j] == 0:
                pheromone_level = pheromone_array[j]
                attractiveness = weights[j]
                probability = ((pheromone_level ** alpha) * (attractiveness ** beta)) / total_probability
                probabilities_for_each_candidate[j] = probability
    
        selected_feature_index = roulette_wheel_selection(probabilities_for_each_candidate, len(probabilities_for_each_candidate))
        print("Selected Feature Index:", selected_feature_index)
        print()
        current_ant[selected_feature_index] = 1
        print("Ant selected:\n", current_ant)
        print()

    return ants_population

# Krok 5 SPRAWDZ FITNESS MRÓWEK
def calculate_ant_fitness(features):
   feature_mask = features.astype(bool)
   X_train_subset = X_train.loc[:, feature_mask]
   X_test_subset = X_test.loc[:, feature_mask]
   gbc.fit(X_train_subset, y_train)
   preds = gbc.predict(X_test_subset)
   f1_score_subset = round(f1_score(y_test, preds, average='weighted'), 3)
   return f1_score_subset

# KROK 6 ZAKTUALIZUJ LISTE FEROMONÓW NA PODSTAWIE WYBORÓW MRÓWEK
def update_pheromones(pheromone_array, ants_population, quality_measure, evaporation_rate=0.1):
    pheromone_array *= (1 - evaporation_rate)
    for ant in ants_population:
        for j in range(len(pheromone_array)):
            if ant[j] == 1:
                # Update pheromone levels based on the quality measure
                pheromone_array[j] += quality_measure[j]
    return pheromone_array

alpha = 1.0
beta = 1.0
ants = generate_ant_population(30, 30)
weights = np.array(mutual_info)
pheromone_array = generate_pheromone_array(30, 0.2)
print("Initial ants population:\n", ants)
print("Wagi:\n",weights)
print("Initial pheromone_array: \n", pheromone_array)

for i in range(5): 
    ants_population_after_move = ants_make_move(ants, pheromone_array=pheromone_array, weights=weights)
    fitness_scores_ants = [calculate_ant_fitness(features) for features in tqdm(ants_population_after_move, desc='Fitness Score Calculation Progress', colour = "green", leave=True)]
    print(max(fitness_scores_ants))
    print(fitness_scores_ants)
    updated_pheromones = update_pheromones(pheromone_array, ants_population_after_move, fitness_scores_ants)
    print("Updated Pheromone Levels:\n", updated_pheromones)
    print("New ants population\n:", ants_population_after_move)

print(np.argmax(fitness_scores_ants))


Ants generated!

Initial ants population:
 [[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 
































Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:09<00:00,  3.31it/s]


0.965
[0.876, 0.936, 0.735, 0.87, 0.693, 0.924, 0.677, 0.919, 0.953, 0.889, 0.918, 0.86, 0.959, 0.866, 0.889, 0.93, 0.872, 0.707, 0.759, 0.65, 0.709, 0.9, 0.942, 0.778, 0.965, 0.906, 0.889, 0.911, 0.877, 0.889]
Updated Pheromone Levels:
 [1.932 1.116 2.385 1.05  0.18  1.104 0.857 2.937 1.133 0.18  0.18  1.04
 2.098 2.778 2.847 3.9   1.052 0.887 0.18  2.13  2.307 1.08  2.064 4.07
 4.04  3.804 0.18  3.824 3.688 1.958]
New ants population
: [[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 
































Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:08<00:00,  3.41it/s]


0.982
[0.935, 0.936, 0.947, 0.936, 0.93, 0.924, 0.959, 0.919, 0.977, 0.913, 0.919, 0.907, 0.941, 0.883, 0.912, 0.953, 0.901, 0.948, 0.872, 0.854, 0.895, 0.936, 0.947, 0.948, 0.982, 0.953, 0.947, 0.942, 0.895, 0.918]
Updated Pheromone Levels:
 [ 5.4788  1.9404  5.9345  1.881   1.092   1.9176  1.7303  9.9953  1.9967
  0.162   0.162   1.843   5.6522  8.6812  5.2983  7.322   1.8478  1.7463
  0.162   4.479   9.2363  1.908   6.5926 12.195   7.564   8.1886  0.162
  9.0936  6.8992  3.5982]
New ants population
: [[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 

...

0.982
[0.918, 0.942, 0.982, 0.935, 0.971, 0.942, 0.947, 0.93, 0.953, 0.918, 0.942, 0.901, 0.953, 0.936, 0.907, 0.947, 0.953, 0.965, 0.942, 0.959, 0.942, 0.942, 0.947, 0.965, 0.977, 0.947, 0.942, 0.942, 0.913, 0.953]
Updated Pheromone Levels:
 [ 9.52092  2.68836 10.25105  5.4329   1.9538   2.66784  2.50427 19.22577
  2.75003  0.1458   0.1458   2.5597   9.85198 15.30108  8.39647 10.3778
  2.61602  2.53667  0.1458   6.9081  21.50067  3.6012  11.61534 27.3805
 11.6926  12.10474  0.1458  15.72024  9.86128  5.14438]
New ants population
: [[0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0
































Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:10<00:00,  2.82it/s]


0.971
[0.918, 0.942, 0.936, 0.947, 0.947, 0.936, 0.953, 0.942, 0.947, 0.907, 0.947, 0.942, 0.942, 0.936, 0.93, 0.953, 0.953, 0.947, 0.925, 0.925, 0.971, 0.947, 0.942, 0.93, 0.936, 0.959, 0.965, 0.948, 0.936, 0.936]
Updated Pheromone Levels:
 [14.994828  3.361524 15.777945 12.46561   2.70542   3.337056  3.206843
 31.433193  3.422027  0.13122   0.13122   3.24573  14.518782 23.130972
 11.276823 13.15202   3.307418  3.230003  0.13122   8.99229  35.857603
  5.13508  18.931806 44.17245  15.20334  16.648266  0.13122  24.576216
 13.555152  6.501942]
New ants population
: [[0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0]
 [0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 
































Fitness Score Calculation Progress: 100%|██████████| 30/30 [00:11<00:00,  2.56it/s]

0.965
[0.918, 0.942, 0.942, 0.959, 0.959, 0.936, 0.948, 0.942, 0.947, 0.912, 0.953, 0.93, 0.953, 0.924, 0.93, 0.942, 0.942, 0.947, 0.925, 0.907, 0.953, 0.942, 0.942, 0.936, 0.936, 0.965, 0.965, 0.942, 0.93, 0.965]
Updated Pheromone Levels:
 [22.6753452  3.9673716 22.6781505 18.891049   3.393878   3.9393504
  3.8341587 46.1878737  4.0268243  0.118098   0.118098   3.851157
 19.7379038 36.5258748 13.8691407 15.604818   3.9186762  3.8540027
  0.118098  10.814061  52.2848427  7.447572  29.2846254 59.411205
 18.363006  22.7034394  0.118098  33.4225944 17.7796368  7.7817478]
New ants population
: [[1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0]
 [0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0]
 [1 1 0 0 0 0 0 1 0 0 0 0